<a href="https://colab.research.google.com/github/Sultan477/DataScience/blob/main/%D0%9F%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D0%B5_%D1%82%D0%B0%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt

### Базовое знакомство с данными

In [ ]:
df = pd.read_csv("data/train.csv")
df.head()

,id,name,tare
0,0,Котлеты МЛМ из говядины 335г,коробка
1,1,Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...,коробка
2,2,"ТВОРОГ (ЮНИМИЛК) ""ПРОСТОКВАШИНО"" ЗЕРНЕНЫЙ 130Г...",стаканчик
3,3,Сыр Плавленый Веселый Молочник с Грибами 190г ...,контейнер
4,4,Жевательный мармелад Маша и медведь буквы 100г,пакет без формы


Какие уникальные значения принимает таргет, есть ли дизбаланс?

In [ ]:
df["tare"].value_counts()

пакет без формы                   9028
бутылка                           7474
коробка                           4196
пакет прямоугольный               3501
обертка                           3217
банка неметаллическая             2238
стаканчик                         2070
банка металлическая               1837
вакуумная упаковка                1071
усадочная упаковка                 993
контейнер                          884
пачка                              691
лоток                              628
туба                               589
гофрокороб                         419
колбасная оболочка                 396
тортница                           324
без упаковки                       322
упаковка с газовым наполнением     289
ведро                              253
ячеистая упаковка                  228
Name: tare, dtype: int64

Разделим выборку на обучающую и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(
    df,
    test_size=0.25,
    stratify=df["tare"]
)

Используем стратифицированное разделение в силу дизбаланса классов.

Делаем мы этого для того, чтобы избежать случайного *выпадания* какого-либо из видов тары. Например, если на трейне к нам не попадут товары, запакованные в тубу, то и на тесте мы их не сможем верно классифицировать. От этого будет страдать обобщающая способность модели. Стратификация позволяет сделать распределение таргетов в трейне и тесте таким, каким оно примерно является в общей совокупности.

Убедимся, что действительно и в трейне, и в тесте схожая доля каждой тары!

In [ ]:
train_shares = df_train["tare"].value_counts() / df_train.shape[0]
test_shares = df_test["tare"].value_counts() / df_test.shape[0]

to_compare = pd.concat((train_shares, test_shares), axis=1)
to_compare.columns = ['Доля в трейне', 'Доля в тесте']
to_compare['Абсолютная разница'] = (to_compare["Доля в трейне"] - \
                                    to_compare["Доля в тесте"]).abs()

to_compare

,Доля в трейне,Доля в тесте,Абсолютная разница
пакет без формы,0.222102,0.222102,0.000000
бутылка,0.183855,0.183920,0.000066
коробка,0.103228,0.103228,0.000000
пакет прямоугольный,0.086138,0.086105,0.000033
обертка,0.079151,0.079118,0.000033
банка неметаллическая,0.055042,0.055107,0.000066
стаканчик,0.050909,0.050974,0.000066
банка металлическая,0.045201,0.045168,0.000033
вакуумная упаковка,0.026340,0.026373,0.000033
усадочная упаковка,0.024437,0.024405,0.000033


### Построим базовую модель в качестве бейзлайна. TF-IDF + KNN

Преобразуем наименования товаров с помощью `tf-idf`, взглянем на результат и ровно на нем обучим простейший `KNN`.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

Импортируем классический `TfidfVectorizer` из `sklearn` и обозначим класс за переменную `tfidf`

In [ ]:
tfidf = TfidfVectorizer()

Произведем `TfIdf` преобразование на первых 5 наименованиях.

Метод `fit_transform` возвращает `sparse matrix`.

Применим метод `toarray`, чтобы получить данные типа `array`.

In [ ]:
tfidf_data = (
    tfidf
    .fit_transform(df["name"].head())
    .toarray()
)

tfidf_data

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.4472136 , 0.        , 0.        , 0.        , 0.        ,
        0.4472136 , 0.        , 0.        , 0.        , 0.4472136 ,
        0.        , 0.4472136 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.4472136 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.30151134, 0.30151134,
        0.        , 0.        , 0.        , 0.        , 0.30151134,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.30151134, 0.        , 0.30151134, 0.30151134, 0.        ,
        0.        , 0.        , 0.30151134, 0.        , 0.        ,
        0.        , 0.60302269, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.4472136 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

Отправим полученный `array` в `DataFrame`, чтобы убедиться в корректности работы метода

In [ ]:
tfidf_data_df = pd.DataFrame(
    tfidf_data,
    index=df["name"].head().index,
    columns=tfidf.get_feature_names_out()
)

tfidf_data_df

,100г,130гр,190г,20,250г,335г,буквы,ванна,веселый,вкуса,...,медведь,мишки,млм,молочник,плавленый,победа,простоквашино,сыр,творог,юнимилк
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.301511,0.301511,0.000000,0.000000,0.000000,0.000000,0.301511,...,0.000000,0.301511,0.000000,0.000000,0.000000,0.603023,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.447214,0.447214
3,0.000000,0.000000,0.377964,0.000000,0.000000,0.000000,0.000000,0.377964,0.377964,0.000000,...,0.000000,0.000000,0.000000,0.377964,0.377964,0.000000,0.000000,0.377964,0.000000,0.000000
4,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Pipeline сам умеет применять `fit_transform`, поэтому можно так компактно записать процесс `tf-idf` преобразования и обучения на нем модели.

Нет необходимости переводить `array` в `DataFrame`, так как модели из `sklearn` умеют отлично работать с np массивами.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

Построим `Pipeline`.

In [ ]:
pipeline_baseline = Pipeline(
    [
        ('tfidf_vectorizer', TfidfVectorizer()),
        ('default_KNN', KNeighborsClassifier())
    ]
)

Зафитим модель тренировочными данными и замерим качество на трейне и тесте.

In [ ]:
pipeline_baseline.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_baseline.predict(df_train["name"])
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_baseline.predict(df_test["name"])
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.898
Accuracy на тестовой выборке составило 0.837


Accuracy даже с учетом дисбаланса классов (максимальная доля около 22%) оказывается достаточно высоким.

Есть смысль повалидироваться на гиперпараметрах модели, так как она может в итоге оказаться финально лучшей.

В качестве параметров для валидации выберем:

- Количество соседей (`n`)
- Способ взвешивания соседей (`weights`)
- Параметр p метрики Минковского (`p`)

In [ ]:
from sklearn.model_selection import GridSearchCV

def gaussian_kernel(distances, h=1):
        return np.exp(- distances**2 / h**2)

parameters_grid = {
    'default_KNN__n_neighbors': [5, 10, 20],
    'default_KNN__weights': ['uniform', 'distance', gaussian_kernel],
    'default_KNN__p': (2, 1),
}

custom_cv = [(df_train.index.to_list(), df_test.index.to_list())]

search_baseline = GridSearchCV(
    pipeline_baseline,
    parameters_grid,
    scoring="accuracy",
    cv=custom_cv,
    verbose=10,
    return_train_score=True
)

search_baseline.fit(df["name"], df["tare"])

Fitting 1 folds for each of 18 candidates, totalling 18 fits
[CV 1/1; 1/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform
[CV 1/1; 1/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=uniform;, score=(train=0.898, test=0.837) total time=   6.8s
[CV 1/1; 2/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=distance
[CV 1/1; 2/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=distance;, score=(train=0.998, test=0.850) total time=   6.5s
[CV 1/1; 3/18] START default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=<function gaussian_kernel at 0x1362e7e50>
[CV 1/1; 3/18] END default_KNN__n_neighbors=5, default_KNN__p=2, default_KNN__weights=<function gaussian_kernel at 0x1362e7e50>;, score=(train=0.974, test=0.854) total time=   6.5s
[CV 1/1; 4/18] START default_KNN__n_neighbors=5, default_KNN__p=1, default_KNN__weights=uniform
[CV 1/1; 4/18] END default_KNN__n_neighbors=

GridSearchCV(cv=[([12619, 22985, 9368, 20471, 29219, 836, 21930, 21138, 20511,
                   38467, 29387, 32935, 25247, 12920, 11015, 27662, 21173,
                   23394, 35592, 31049, 19076, 12119, 20934, 14739, 19282, 7561,
                   21490, 20145, 17016, 35524, ...],
                  [36160, 16249, 38619, 7152, 33356, 3329, 34481, 31220, 1680,
                   36759, 38990, 14533, 38448, 25028, 2886, 1393, 31922, 15256,
                   30898, 28602, 31556, 28276, 3955, 18086, 12963, 3912, 19638,
                   23368, 26803, 13988, ...])],
             estimator=Pipeline(steps=[('tfidf_vectorizer', TfidfVectorizer()),
                                       ('default_KNN',
                                        KNeighborsClassifier())]),
             param_grid={'default_KNN__n_neighbors': [5, 10, 20],
                         'default_KNN__p': (2, 1),
                         'default_KNN__weights': ['uniform', 'distance',
                                 

Взглянем на лучшую модель и ее качество.

In [ ]:
print(f"Best parameter (CV score={search_baseline.best_score_:.5f}):")
print(search_baseline.best_params_)

Best parameter (CV score=0.86459):
{'default_KNN__n_neighbors': 20, 'default_KNN__p': 1, 'default_KNN__weights': <function gaussian_kernel at 0x1362e7e50>}


In [ ]:
pipeline_baseline.set_params(**search_baseline.best_params_)

Pipeline(steps=[('tfidf_vectorizer', TfidfVectorizer()),
                ('default_KNN',
                 KNeighborsClassifier(n_neighbors=20, p=1,
                                      weights=<function gaussian_kernel at 0x1362e7e50>))])

Дополнительно проверим качество лучшей модели (помимо логов `gridsearch`)

In [ ]:
pipeline_baseline.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_baseline.predict(df_train["name"])
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_baseline.predict(df_test["name"])
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.997
Accuracy на тестовой выборке составило 0.865


Качество на трейне выросло, стало почти идеальным +0.099

Качество на тесте тоже выросло, хоть и не так сильно +0.029

Очевидно в глаза бросается переобучение, поэтому есть смысл протестировать модели, менее склонные к нему.

### SVM, RandomForest

Построим пайплайны с тремя предложенными к рассмотрению моделями

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

pipeline_svm = Pipeline(
    [
        ('tfidf_vectorizer', TfidfVectorizer()),
        ('SVC', SVC())
    ]
)

pipeline_rf = Pipeline(
    [
        ('tfidf_vectorizer', TfidfVectorizer()),
        ('RF', RandomForestClassifier())
    ]
)

Найдем лучшие гиперпараметры для `SVM` и оценим качество на трейне/тесте

In [ ]:
svm_parameters_grid = {
    'SVC__C': [1, 0.5, 3],
    'SVC__kernel': ['linear', 'rbf', 'sigmoid']
}

search_svm = GridSearchCV(
    pipeline_svm,
    svm_parameters_grid,
    scoring="accuracy",
    cv=custom_cv,
    return_train_score=True
)

search_svm.fit(df["name"], df["tare"])

pipeline_svm.set_params(**search_svm.best_params_)

pipeline_svm.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_svm.predict(df_train["name"])
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_svm.predict(df_test["name"])
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.997
Accuracy на тестовой выборке составило 0.874


Качество на тесте выросло +0.01 по сравнению с лучшим KNN

Возможно, стоило лучше поиграться с гиперпараметрыми, например, с `penalty`.

Найдем лучшие гиперпараметры для `RandomForest` и оценим качество на трейне/тесте

In [ ]:
rf_parameters_grid = {
    'RF__n_estimators': [10, 100, 200],
    'RF__max_depth': [5, 15, 30, None]
}

search_rf = GridSearchCV(
    pipeline_rf,
    rf_parameters_grid,
    scoring="accuracy",
    cv=custom_cv,
    return_train_score=True
)

search_rf.fit(df["name"], df["tare"])

pipeline_rf.set_params(**search_rf.best_params_)

pipeline_rf.fit(
    df_train["name"],
    df_train["tare"]
)

train_preds = pipeline_rf.predict(df_train["name"])
train_accuracy = np.mean(train_preds == df_train["tare"].values)

test_preds = pipeline_rf.predict(df_test["name"])
test_accuracy = np.mean(test_preds == df_test["tare"].values)

print(f"Accuracy на тренировочной выборке составило {np.round(train_accuracy, decimals=3)}")
print(f"Accuracy на тестовой выборке составило {np.round(test_accuracy, decimals=3)}")

Accuracy на тренировочной выборке составило 0.998
Accuracy на тестовой выборке составило 0.831


Случайный лес справляется хуже.

Неудивительно, что модели обучаются очень долго, так как tf-idf по большому корпусу текстов содержит $n \cdot m$ элементов, где $n$ - количество текстов, $m$ - количество уникальных слов.

### Гипотеза: сузив tf-idf пространство, можно получить лучшее качество SVM

Идея следующая: сузим пространство с помощью PCA преобразования, выделим в новом пространстве несколько кластеров (скажем, 15 штук: хотя на этом параметре можно еще повалидироваться), после чего каждый объект подменим вектором его расстояний до центра из каждого кластера.

У такого сложного преобразования есть некоторая интерпретация: мы выделяем наиболее важные компоненты каждого объекта, находим некоторые "общий паттерны" по аналогии с жанрами фильмов, после чего получаем каждый объект как смесь каждого "жанра".

Так как нам важна обобщающая способность, алгоритмы преобразований (PCA + Kmeans) будем фитить исключительно в тренировочную часть.

Начнем с выделения оригинальных векторов tf-idf.

In [ ]:
tfidf_ = TfidfVectorizer()
tfidf_.fit(df_train["name"])

tfidf_array_train = (
    tfidf_
    .transform(df_train["name"])
    .toarray()
)

tfidf_array_test = (
    tfidf_
    .transform(df_test["name"])
    .toarray()
)


Выделим 20 главных компонент. Не забудем центрировать данные на всякий случай, хоть sklearn и делает это за нас.

In [ ]:
from sklearn.decomposition import PCA

centered_train = tfidf_array_train - tfidf_array_train.mean()
centered_test = tfidf_array_test - tfidf_array_test.mean()

pca = PCA(n_components=20)
pca.fit(centered_train)

pca_train = pca.transform(centered_train)
pca_test = pca.transform(centered_test)

Выделим 15 кластеров в новом полученном множестве (объекты те же, только признаков теперь всего 20) и замерим расстояние от каждого объекта до центра каждого кластера.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=15, random_state=0)
kmeans.fit(pca_train)

dists_columns = ['DistanceTo1thCluster',
                 'DistanceTo2thCluster',
                 'DistanceTo3thCluster',
                 'DistanceTo4thCluster',
                 'DistanceTo5thCluster',
                 'DistanceTo6thCluster',
                 'DistanceTo7thCluster',
                 'DistanceTo8thCluster',
                 'DistanceTo9thCluster',
                 'DistanceTo10thCluster',
                 'DistanceTo11thCluster',
                 'DistanceTo12thCluster',
                 'DistanceTo13thCluster',
                 'DistanceTo14thCluster',
                 'DistanceTo15thCluster']

dists_df_train = pd.DataFrame(
    data=kmeans.transform(pca_train),
    columns=dists_columns
)

dists_df_test = pd.DataFrame(
    data=kmeans.transform(pca_test),
    columns=dists_columns
)

dists_df_train.head()

,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,0.333051,0.508755,0.512241,0.433715,0.507693,0.299089,0.447958,0.425490,0.427266,0.262639,0.506685,0.435924,0.410349,0.476858,0.463900
1,0.270865,0.471032,0.477136,0.393654,0.468545,0.392705,0.418159,0.392699,0.150807,0.371305,0.470058,0.382083,0.371613,0.451706,0.434965
2,0.129637,0.407601,0.422208,0.332193,0.414372,0.316636,0.350791,0.276739,0.358744,0.275885,0.410634,0.315394,0.300338,0.398919,0.367578
3,0.318109,0.503839,0.510207,0.402907,0.494484,0.410380,0.134430,0.427619,0.458188,0.360578,0.496030,0.424072,0.397136,0.487527,0.456718
4,0.147730,0.405791,0.420199,0.295004,0.409420,0.301258,0.342247,0.312124,0.355722,0.299204,0.402431,0.136371,0.281426,0.389909,0.356471


Обучим SVM

In [ ]:
from sklearn.linear_model import SGDClassifier

SGD = SGDClassifier(max_iter=30000)

train_acc = []
test_acc = []

for alpha in np.linspace(1e-6, 1, 5):
    SGD.set_params(**{'alpha': alpha})

    SGD.fit(
        dists_df_train,
        df_train['tare']
    )

    train_preds = SGD.predict(dists_df_train)
    train_accuracy = np.mean(train_preds == df_train["tare"].values)
    train_acc.append(train_accuracy)

    test_preds = SGD.predict(dists_df_test)
    test_accuracy = np.mean(test_preds == df_test["tare"].values)
    test_acc.append(test_accuracy)

In [ ]:
test_acc

[0.39667388309387913,
 0.37679590631765403,
 0.29462704192088174,
 0.35977169848455026,
 0.2901003739421374]

In [ ]:
train_acc

[0.3964770714426294,
 0.37554943252640555,
 0.2960703273633799,
 0.35763957226267795,
 0.29292134094338385]

Качество сильно ухудшилось! Такое произошло вероятно из-за того, что алгоритм преобразования не подходит под наши данные.

### Улучшим процедуру tf-idf преобразования: предварительный стемминг

In [ ]:
from TextProcessing import preprocessing

Взглянем на результат обработки

In [ ]:
df["name"].head()

0                         Котлеты МЛМ из говядины 335г
1    Победа Вкуса конфеты Мишки в лесу 250г(КФ ПОБЕ...
2    ТВОРОГ (ЮНИМИЛК) "ПРОСТОКВАШИНО" ЗЕРНЕНЫЙ 130Г...
3    Сыр Плавленый Веселый Молочник с Грибами 190г ...
4      Жевательный мармелад Маша и медведь  буквы 100г
Name: name, dtype: object

In [ ]:
df["name"].head().apply(preprocessing)

0                         котлет млм из говядин 335г
1     побед вкус конфет мишк в лес 250г кф побед  20
2    творог  юнимилк   простоквашин  зернен 130гр 7 
3         сыр плавлен весел молочник с гриб 190г ван
4         жевательн мармелад маш и медвед  букв 100г
Name: name, dtype: object

В ходе экспериментов данное преобразование никак не улучшило существующую модель.

### В какую сторону можно искать улучшения?

Во-первых, необходимо лучше обработать текст перед тем, как скармливать его `tf-idf`. Например, такие сущности как 400гр и 0.4кг можно преобразовать к единому формату, то же касается мер объема. Также некоторые названия могут писаться слитно, например, ?*КолбасаДокторская*. В таком случае `tf-idf` распознает это как отдельное уникальное слово, скорее непохожее на просто Колбасу.

Во-вторых, можно продолжить эксперименты с моделями и посмотреть побольше в сторону ансамблей и метрических алгоритмов поверх tf-idf. Или сильнее и глубже поиграться с параметрами регуляризации того же SVM.

Наконец, есть множество других способов классификации текстов: нейросетевой подход, LDA, etc.